# Importing required packages

In [1]:
!pip install --upgrade langchain  -q    #This flag tells pip to run in quiet mode, suppressing output messages. This keeps the code block cleaner and avoids cluttering the console.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [2]:
!pip install sentence_transformers -q     # provides pre-trained models and tools for text embedding. It allows you to convert sentences or text into numerical representations that can be used for various tasks like semantic search, text classification, and information retrieval.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.5 MB/s eta 0:00:00


In [3]:
!pip install unstructured -q    #This package offers tools and utilities for working with unstructured data, such as text, images, and audio
!pip install unstructured[local-inference] -q     #This command installs the unstructured package along with the local-inference extra. This extra installs additional dependencies required to run some functionalities of the unstructured package locally on your machine, such as performing inference on models without needing a remote server.
# !pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q       # installs detectron2 (v0.6) from GitHub, a framework for building and training computer vision models like object detection and image segmentation. #egg=detectron2: This tells pip to create an installable package named detectron2 from the downloaded source code.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.2 MB/s eta 0:00:00
  Pre

In [ ]:
!apt-get install poppler-utils          # Poppler-utils is a collection of command-line tools based on the Poppler library  for reading and writing PDF documents. It provides a set of tools and libraries that can be used to extract text, images, and metadata from PDF files.

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (418 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123595 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


# 1. Loading from File Upload

In [4]:
!pip install PyMuPDF  # for pdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 48.3 MB/s eta 0:00:00


In [5]:
from google.colab import files
import fitz  # PyMuPDF
import os

def upload_pdf():
    # Upload the file
    uploaded_file = files.upload()

    # Get the filename
    for fn in uploaded_file.keys():
        print(f'User uploaded file "{fn}" with length {len(uploaded_file[fn])} bytes')

        # Save the uploaded file to the local environment
        with open(fn, 'wb') as f:
            f.write(uploaded_file[fn])

        file_path = os.path.abspath(fn)
        print(f"File uploaded successfully to: {file_path}")

        return file_path

# Upload the PDF file and get the file path
file_path = upload_pdf()


Saving Nepal.pdf to Nepal.pdf
User uploaded file "Nepal.pdf" with length 183715 bytes
File uploaded successfully to: /content/Nepal.pdf


In [6]:
!pip install -U langchain-community       # langchain-community is a collection of community-contributed modules and extensions for the langchain library. These modules can provide additional functionalities or integrations for various use cases.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.4 MB/s eta 0:00:00


# 2. Splitting documents

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=500,chunk_overlap=20):
  with fitz.open(file_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_text(text)
  return docs

# If a file was uploaded, split it into chunks
if file_path:
    docs = split_docs(file_path)
    print(len(docs))

5


In [ ]:
print(docs[2])

below the poverty line. Political instability has hindered economic development and social 
progress. Additionally, Nepal is prone to natural disasters such as earthquakes, landslides, and 
floods, which can have devastating consequences. 
Despite these challenges, Nepal is a resilient nation with a rich cultural heritage. The country is 
home to diverse ethnic groups, each with its own unique language, customs, and traditions. Nepali


# 3. Creating embeddings

In [8]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
query_result = embeddings.embed_query("Welcome to Chatbot")
len(query_result) # print the length of the embedding vector

384

# 4. Storing embeddings in Pinecone

In [24]:
!pip install --upgrade pinecone

In [30]:
!pip install langchain_pinecone

In [10]:
# !pip install --upgrade pinecone-client -q
!pip install --upgrade pinecone-client[grpc] -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [11]:
# from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec

# # Initialize Pinecone with the gRPC client
# pc = Pinecone(api_key='485df547-324f-4bd6-97d5-98c480140498')

# # Check if the index exists; if not, create it
# index_name = "chatbot2"
# try:
#   if index_name not in pc.list_indexes().names():
#     pc.create_index(
#         name=index_name,
#         dimension=384,  # Dimension of your embeddings
#         metric="euclidean",
#         spec=ServerlessSpec(
#             cloud="aws",
#             region="us-east-1"
#         )
#     )
#   print("Index creation or verification successful.")
# except Exception as e:
#    print("Error:", e)


Index creation or verification successful.


In [33]:
# from pinecone.grpc import PineconeGRPC as Pinecone #if needed comment
from pinecone import ServerlessSpec
from pinecone import Pinecone #added
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.schema import Document
from langchain_pinecone import PineconeVectorStore

from google.colab import userdata
api_key= userdata.get('chat1_key')

pc = Pinecone(api_key=api_key)

index_name = "chatbot1"

# Check if the index exists; if not, create it
# if index_name not in pc.list_indexes():
#     pc.create_index(
#         name=index_name,
#         dimension=384,  # Dimension of your embeddings
#         metric="euclidean",
#         spec=ServerlessSpec(
#             cloud="aws",
#             region="us-east-1"
#         )
#     )

# Convert docs (list of strings) to Document objects
documents = [Document(page_content=doc) for doc in docs]

# Use the Langchain integration with Pinecone
index = PineconeVectorStore.from_documents(documents, embeddings, index_name=index_name)

print("Index creation and document insertion completed successfully.")

PineconeConfigurationError: You haven't specified an Api-Key.

# 5. Access and search embeddings using the similarity_search

In [ ]:
def get_similiar_docs(query,k=1,score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query,k=k)
  else:
    similar_docs = index.similarity_search(query,k=k)
  return similar_docs

query = "How is Nepal economy"
similar_docs = get_similiar_docs(query)
similar_docs
